In [7]:
from load_database import load_vector_store_local

vecstore = load_vector_store_local("sec-10-K-finbert","10-K",if_finbert=True)

In [8]:
import chromadb

query_results = vecstore.query(
    query_texts="What are the risk factors of Apple for 2022?",
    n_results=20,
    where={'full_metadata':'AAPL_2022_RISK_FACTORS_10-K'},
    include=["metadatas","documents","distances","embeddings"]
)

In [9]:
query_results

{'ids': [['id99',
   'id20',
   'id33',
   'id165',
   'id104',
   'id201',
   'id96',
   'id102',
   'id162',
   'id110',
   'id54',
   'id80',
   'id198',
   'id169',
   'id155',
   'id194',
   'id199',
   'id205',
   'id197',
   'id196']],
 'embeddings': [[[-0.1222430020570755,
    0.7765983939170837,
    -0.2157733142375946,
    -0.12686049938201904,
    0.5043826699256897,
    -0.1835414171218872,
    -0.06209033355116844,
    0.46575111150741577,
    0.20971530675888062,
    0.1265983283519745,
    -0.030225718393921852,
    -0.5532909631729126,
    0.4046131670475006,
    0.25730106234550476,
    0.08405459672212601,
    0.29521995782852173,
    0.08178349584341049,
    0.11370395869016647,
    -0.033475685864686966,
    0.12572599947452545,
    -0.1779337078332901,
    0.17947839200496674,
    -0.08164738118648529,
    0.680227518081665,
    0.5198003649711609,
    -0.26727476716041565,
    0.0978279709815979,
    0.44198551774024963,
    0.16711457073688507,
    0.288491040468

In [11]:
import re

text = "This is a sentence. This is another sentence with a decimal number 3.14. Here is a third sentence."

# Regular expression to match sentence endings while excluding decimal numbers
pattern = r'\.\s'

sentences = re.split(pattern, text)

for i, sentence in enumerate(sentences, start=1):
    print(f"Sentence {i}: {sentence.strip()}")


Sentence 1: This is a sentence
Sentence 2: This is another sentence with a decimal number 3.14
Sentence 3: Here is a third sentence.


In [3]:
len(query_results['embeddings'][0])

20

In [4]:
query_results.keys()

dict_keys(['ids', 'embeddings', 'documents', 'metadatas', 'distances'])

In [5]:
from chromadb.utils import embedding_functions


default_ef = embedding_functions.DefaultEmbeddingFunction()

query_embed = default_ef(texts="What are the risk factors of Apple for 2022?")

In [6]:
# query_embed

In [14]:
llm1_output_dict = {'Section_Names': ['RISK_FACTORS',
  'MARKET_RISK_DISCLOSURES',
  'LEGAL_PROCEEDINGS',
  'UNRESOLVED_STAFF_COMMENTS',
  'MINE_SAFETY'],
 'Tickers': ['AAPL', 'TSLA'],
 'Years': ['2021', '2022'],
 'augmented_query': ['What is the risk associated with Apple stock in 2021?',
  'What is the risk associated with Tesla stock in 2021?',
  'What is the risk associated with Apple stock in 2022?',
  'What is the risk associated with Tesla stock in 2022?']}

In [8]:
s = "TSLA_2023_RISK_FACTORS_10-K"

In [9]:
import itertools
section_names = llm1_output_dict['Section_Names']
ticker_names = llm1_output_dict['Tickers']
years = llm1_output_dict['Years']
doc_name = "10-K"
individual_metadata = [elem[0]+"_"+elem[1]+"_"+elem[2]+"_"+doc_name for elem in list(itertools.product(ticker_names,years,section_names))]

In [10]:
individual_metadata

['AAPL_2021_RISK_FACTORS_10-K',
 'AAPL_2021_MARKET_RISK_DISCLOSURES_10-K',
 'AAPL_2021_LEGAL_PROCEEDINGS_10-K',
 'AAPL_2021_UNRESOLVED_STAFF_COMMENTS_10-K',
 'AAPL_2021_MINE_SAFETY_10-K',
 'AAPL_2022_RISK_FACTORS_10-K',
 'AAPL_2022_MARKET_RISK_DISCLOSURES_10-K',
 'AAPL_2022_LEGAL_PROCEEDINGS_10-K',
 'AAPL_2022_UNRESOLVED_STAFF_COMMENTS_10-K',
 'AAPL_2022_MINE_SAFETY_10-K',
 'TSLA_2021_RISK_FACTORS_10-K',
 'TSLA_2021_MARKET_RISK_DISCLOSURES_10-K',
 'TSLA_2021_LEGAL_PROCEEDINGS_10-K',
 'TSLA_2021_UNRESOLVED_STAFF_COMMENTS_10-K',
 'TSLA_2021_MINE_SAFETY_10-K',
 'TSLA_2022_RISK_FACTORS_10-K',
 'TSLA_2022_MARKET_RISK_DISCLOSURES_10-K',
 'TSLA_2022_LEGAL_PROCEEDINGS_10-K',
 'TSLA_2022_UNRESOLVED_STAFF_COMMENTS_10-K',
 'TSLA_2022_MINE_SAFETY_10-K']

In [11]:
from load_database import load_vector_store_local
from utils import maximal_marginal_relevance
vecstore = load_vector_store_local("sec-10-K","10-K")

In [11]:
from chromadb.utils import embedding_functions
query = "What are the risk Factors of Apple and Tesla for the year 2022 and 2021?"


default_ef = embedding_functions.DefaultEmbeddingFunction()

query_embed = default_ef(texts=query)
# query_embed

In [12]:
import numpy as np
query_embed_arr = np.array(query_embed[0])
query_embed_arr.shape

(384,)

In [28]:
from collections import defaultdict
all_relevant_docs = defaultdict()
for tic in ticker_names:
    for yr in years:
        individual_metadata = [{"full_metadata":elem[0]+"_"+elem[1]+"_"+elem[2]+"_"+doc_name} for elem in list(itertools.product([tic],[yr],section_names))]
        where_clause = {"$or":individual_metadata}
        query_results = vecstore.query(
        query_texts=query,
        n_results=20,
        where=where_clause,
        include=["metadatas","documents","distances","embeddings"]
        )
        idxs = maximal_marginal_relevance(query_embed_arr,query_results['embeddings'][0],k=5)
        relevant_docs = [query_results['documents'][0][i] for i in idxs]
        relevant_metadata = [ query_results['metadatas'][0][i] for i in idxs]
        print(len(relevant_docs),len(relevant_metadata))
        all_relevant_docs[f"{tic}_{yr}"] = {"docs":relevant_docs,"metadata":relevant_metadata}

5 5
5 5
5 5
5 5


In [18]:
all_relevant_docs.keys()

dict_keys(['AAPL_2021', 'AAPL_2022', 'TSLA_2021', 'TSLA_2022'])

In [43]:
# query_results['embeddings'][0]

In [44]:
idxs = maximal_marginal_relevance(query_embed_arr,query_results['embeddings'][0],k=5)
idxs

[12, 14, 19, 11, 7]

In [53]:
relevant_docs = [query_results['documents'][0][i] for i in idxs]
relevant_metadata = [ query_results['metadatas'][0][i] for i in idxs]

In [54]:
relevant_metadata

[{'full_metadata': 'AAPL_2022_RISK_FACTORS_10-K'},
 {'full_metadata': 'AAPL_2022_RISK_FACTORS_10-K'},
 {'full_metadata': 'AAPL_2022_RISK_FACTORS_10-K'},
 {'full_metadata': 'AAPL_2022_RISK_FACTORS_10-K'},
 {'full_metadata': 'AAPL_2022_RISK_FACTORS_10-K'}]

In [24]:
for k,v in all_relevant_docs.items():
    print(k,v)
    break

AAPL_2021 {'docs': 'The Company’s business, results of operations and financial condition, as well as the price of the Company’s stock, have been adversely affected and could in the future be materially adversely affected by the COVID-19 pandemic.\n COVID-19 has had, and continues to have, a significant impact around the world, prompting governments and businesses to take unprecedented measures in response. The Company’s minority market share in the global smartphone, personal computer and tablet markets can make developers less inclined to develop or upgrade software for the Company’s products and more inclined to devote their resources to developing and upgrading software for competitors’ products with larger market share.\n When developers focus their efforts on these competing platforms, the availability and quality of applications for the Company’s devices can suffer. These new ventures are inherently risky and may not be successful.\n The failure of any significant investment cou

In [30]:
def get_relevant_docs_via_mmr(relevant_docs_dict:dict):
    relevant_sentences = ""

    for key,value in relevant_docs_dict.items():
        tic,year = key.split("_")
        relevant_sentences+=f"Relevant documents for {tic} in the year {year}"+": "
        print(value)
        relevant_sentences+="\n".join(value['docs'])
        relevant_sentences+="\n\n"
    
    return relevant_sentences

len(get_relevant_docs_via_mmr(all_relevant_docs).split(" "))

{'docs': ['The Company’s business, results of operations and financial condition, as well as the price of the Company’s stock, have been adversely affected and could in the future be materially adversely affected by the COVID-19 pandemic.\n COVID-19 has had, and continues to have, a significant impact around the world, prompting governments and businesses to take unprecedented measures in response.', 'The Company’s minority market share in the global smartphone, personal computer and tablet markets can make developers less inclined to develop or upgrade software for the Company’s products and more inclined to devote their resources to developing and upgrading software for competitors’ products with larger market share.\n When developers focus their efforts on these competing platforms, the availability and quality of applications for the Company’s devices can suffer.', 'These new ventures are inherently risky and may not be successful.\n The failure of any significant investment could 

1271

In [31]:
from utils import key_modifier,modify_all_keys,get_query_metadata,get_relevant_sentences,maximal_marginal_relevance


In [1]:
from load_database import get_relevant_dict_with_mmr,get_relevant_docs_via_mmr
from load_database import load_vector_store_local
from utils import maximal_marginal_relevance
vecstore = load_vector_store_local("sec-10-K","10-K")


/home/athekunal/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name /home/athekunal/.cache/torch/sentence_transformers/ProsusAI_finbert. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/athekunal/.cache/torch/sentence_transformers/ProsusAI_finbert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassifica

In [2]:
from collections import defaultdict
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings
from chromadb.utils import embedding_functions
import numpy as np
import itertools

In [15]:

def get_relevant_dict_with_mmr_(llm1_output_dict,restore_collection,user_query,doc_name="10-K"):
    section_names = llm1_output_dict['Section_Names']
    ticker_names = llm1_output_dict['Tickers']
    years = llm1_output_dict['Years']
    all_relevant_docs = defaultdict()
    default_ef = embedding_functions.DefaultEmbeddingFunction()
    query_embed = default_ef(texts=user_query)
    query_embed_arr = np.array(query_embed[0])

    for tic in ticker_names:
        for yr in years:
            individual_metadata = [{"full_metadata":elem[0]+"_"+elem[1]+"_"+elem[2]+"_"+doc_name} for elem in list(itertools.product([tic],[yr],section_names))]
            print(individual_metadata)
            where_clause = {"$or":individual_metadata}
            print(where_clause)
            query_results = restore_collection.query(
            query_texts=user_query,
            n_results=20,
            where=where_clause,
            include=["metadatas","documents","distances","embeddings"]
            )
            print(query_results)
            idxs = maximal_marginal_relevance(query_embed_arr,query_results['embeddings'][0],k=5)
            relevant_docs = [query_results['documents'][0][i] for i in idxs]
            relevant_metadata = [ query_results['metadatas'][0][i] for i in idxs]
            all_relevant_docs[f"{tic}_{yr}"] = {"docs":relevant_docs,"metadata":relevant_metadata}

    return all_relevant_docs

In [16]:
print(vecstore)

name='SEC-10-K' id=UUID('68fc286d-f5c1-45c4-893b-58f73a6a7e3c') metadata=None


In [8]:
llm1_output_dict = {'Section_Names': ['RISK_FACTORS', 'MARKET RISK DISCLOSURES', 'LEGAL PROCEEDINGS', 'UNRESOLVED STAFF COMMENTS', 'MINE SAFETY'], 'Tickers': ['AAPL'], 'Years': ['2021', '2022'], 'augmented_query': ['Compare the risk associated with Apple stock for the year 2021', 'Compare the risk associated with Apple stock for the year 2022']}
relevant_dict = get_relevant_dict_with_mmr_(llm1_output_dict,vecstore,"Compare the risk associated with Apple stock for the years 2021 and 2022?")
print(relevant_dict)


{'$or': [{'full_metadata': 'AAPL_2021_RISK FACTORS_10-K'}, {'full_metadata': 'AAPL_2021_MARKET RISK DISCLOSURES_10-K'}, {'full_metadata': 'AAPL_2021_LEGAL PROCEEDINGS_10-K'}, {'full_metadata': 'AAPL_2021_UNRESOLVED STAFF COMMENTS_10-K'}, {'full_metadata': 'AAPL_2021_MINE SAFETY_10-K'}]}
{'ids': [[]], 'embeddings': [[]], 'documents': [[]], 'metadatas': [[]], 'distances': [[]]}
{'$or': [{'full_metadata': 'AAPL_2022_RISK FACTORS_10-K'}, {'full_metadata': 'AAPL_2022_MARKET RISK DISCLOSURES_10-K'}, {'full_metadata': 'AAPL_2022_LEGAL PROCEEDINGS_10-K'}, {'full_metadata': 'AAPL_2022_UNRESOLVED STAFF COMMENTS_10-K'}, {'full_metadata': 'AAPL_2022_MINE SAFETY_10-K'}]}
{'ids': [[]], 'embeddings': [[]], 'documents': [[]], 'metadatas': [[]], 'distances': [[]]}
defaultdict(None, {'AAPL_2021': {'docs': [], 'metadata': []}, 'AAPL_2022': {'docs': [], 'metadata': []}})


In [ ]:
relevant_sentences = get_relevant_docs_via_mmr(relevant_dict)
print(relevant_sentences)

In [17]:
# user_query = "Compare the risk associated with Apple stock for the years 2021 and 2022?"
# vecstore.query(
#     query_texts=user_query,
#     n_results=2,
#     include=["metadatas","documents","distances","embeddings"]
#     )

In [20]:
"_".join("RISK_FACTORS".split(" "))

'RISK_FACTORS'

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from chromadb.api.types import Documents, EmbeddingFunction, Embeddings

finbert_embed = HuggingFaceEmbeddings(model_name="ProsusAI/finbert")

class FinBertEmbeddings(EmbeddingFunction):
    def __call__(self, texts: Documents) -> Embeddings:
        embed_out = finbert_embed.embed_documents(texts)
        return embed_out

No sentence-transformers model found with name /home/athekunal/.cache/torch/sentence_transformers/ProsusAI_finbert. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/athekunal/.cache/torch/sentence_transformers/ProsusAI_finbert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
query = "What are the risk factors Apple for the year 2022?"

finbert = FinBertEmbeddings()

In [7]:
import numpy as np
query_embed = finbert(texts=query)
query_embed_arr = np.array(query_embed[0])

In [8]:
query_embed_arr

array([ 1.82527483e-01,  2.10601196e-01, -2.58634448e-01, -2.08617985e-01,
       -9.78663191e-02, -7.12072432e-01,  7.79373944e-01,  6.05757713e-01,
        2.57762700e-01, -1.62297443e-01, -4.09485430e-01, -4.43593144e-01,
        3.94873530e-01,  2.59973437e-01,  1.37500418e-02, -4.95217331e-02,
        3.22281986e-01,  4.67214704e-01,  3.84518653e-01,  7.69835338e-02,
        3.88191104e-01,  1.59892544e-01,  2.74140090e-02,  2.96534538e-01,
       -2.21730903e-01,  7.43004322e-01,  1.94182739e-01, -3.12091261e-01,
        7.63344467e-02,  5.54689407e-01,  6.61128461e-01,  3.79334718e-01,
       -2.87863642e-01,  4.58972663e-01,  5.86117089e-01, -4.76604313e-01,
        1.56925365e-01, -2.43099127e-02, -4.13171202e-01,  8.34424123e-02,
       -7.89192855e-01, -2.88674563e-01,  3.10100198e-01, -5.59933841e-01,
        6.27205595e-02, -7.55767524e-01,  2.88778514e-01, -3.42592783e-02,
       -5.05156994e-01,  5.89336693e-01, -8.60973299e-01, -1.30691871e-01,
       -1.29021347e-01,  

In [16]:
from collections import defaultdict
relevant_dict = defaultdict(None, {'AMZN_2021': {'docs': ['In arrangements where we have multiple performance obligations, the transaction price is allocated to each performance obligation using the relative stand-alone selling price.\nWe generally determine stand-alone selling prices based on the prices charged to customers or using expected cost plus a margin.\nA description of our principal revenue generating activities is as follows: We offer consumer products through our online and physical stores.', 'Our total long-term debt obligations are as follows (in millions): Unamortized discount and issuance costs, net The weighted average remaining lives of the 2012, 2014, 2017, and 2020 Notes were 1.9, 11.8, 16.2, and 18.7 years as of December 31, 2020.\nThe combined weighted average remaining life of the Notes was 15.8 years as of December 31, 2020.\nInterest on the Notes is payable semi-annually in arrears.', 'Amounts held in foreign currencies were $15.3 billion and $23.5 billion as of December 31, 2019 and 2020, and were primarily Euros, British Pounds, and Japanese Yen.\nCash provided by (used in) operating activities was $38.5 billion and $66.1 billion in 2019 and 2020.', 'Our federal tax provision included the election of full expensing of qualified property for 2018 and 2019 and a partial election for 2020.\nThe components of the provision for income taxes, net are as follows (in millions): U.S.', 'We have organized our operations into three segments: North America, International, and AWS.\nSee “Note 10 — Segment Information.” Certain prior period amounts have been reclassified to conform to the current period presentation.\n“Proceeds from short-term debt, and other” were reclassified from “Proceeds from long-term debt and other” and “Repayments of short-term debt, and other” were reclassified from “Repayments of long-term debt and other” on our consolidated statements of cash flows.', 'We are subject to income taxes in the U.S.\n(federal and state) and numerous foreign jurisdictions.\nSignificant judgment is required in evaluating our tax positions and determining our provision for income taxes.\nDuring the ordinary course of business, there are many transactions and calculations for which the ultimate tax determination is uncertain.\nWe establish reserves for tax-related uncertainties based on estimates of whether, and the extent to which, additional taxes will be due.', 'Regulation G, Conditions for Use of Non-GAAP Financial Measures, and other SEC regulations define and prescribe the conditions for use of certain non-GAAP financial information.\nOur measures of free cash flows and the effect of foreign exchange rates on our consolidated statements of operations meet the definition of non-GAAP financial measures.', 'The functional currency of our subsidiaries that either operate or support these stores is generally the same as the local currency.\nAssets and liabilities of these subsidiaries are translated into U.S.\nDollars at period-end foreign exchange rates, and revenues and expenses are translated at average rates prevailing throughout the period.', 'Unearned revenue is recorded when payments are received or due in advance of performing our service obligations and is recognized over the service period.\nUnearned revenue primarily relates to prepayments of AWS services and Amazon Prime memberships.\nOur total unearned revenue as of December 31, 2019 was $10.2 billion, of which $7.9 billion was recognized as revenue during the year ended December 31, 2020 and our total unearned revenue as of December 31, 2020 was $11.6 billion.', 'There were no repurchases of common stock in 2018, 2019, or 2020.\nEmployees vest in restricted stock unit awards over the corresponding service term, generally between two and five years.\nStock-based compensation expense is as follows (in millions): The related tax benefits were $1.1 billion, $1.4 billion, and $1.9 billion for 2018, 2019, and 2020.'], 'metadata': [{'full_metadata': 'AMZN_2021_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2021_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2021_MANAGEMENT_DISCUSSION_10-K'}, {'full_metadata': 'AMZN_2021_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2021_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2021_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2021_MANAGEMENT_DISCUSSION_10-K'}, {'full_metadata': 'AMZN_2021_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2021_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2021_FINANCIAL_STATEMENTS_10-K'}]}, 'AMZN_2022': {'docs': ['During 2019, 2020, and 2021, the fair value of restricted stock units that vested was $11.7 billion, $15.5 billion, and $18.2 billion.\nAs of December 31, 2021, common stock available for future issuance to employees is 97 million shares.\nCertain foreign subsidiary earnings and losses are subject to current U.S.\ntaxation and the subsequent repatriation of those earnings is not subject to tax in the U.S.\nThe U.S.', 'In arrangements where we have multiple performance obligations, the transaction price is allocated to each performance obligation using the relative stand-alone selling price.\nWe generally determine stand-alone selling prices based on the prices charged to customers or using expected cost plus a margin.\nA description of our principal revenue generating activities is as follows: We offer consumer products through our online and physical stores.', 'We are not subject to any financial covenants under the Notes.\nThe estimated fair value of the Notes was approximately $37.7 billion and $53.3 billion as of December 31, 2020 and 2021, which is based on quoted prices for our debt as of those dates.\nAs of December 31, 2021, future principal payments for our total long-term debt were as follows (in millions): We have U.S.', 'Regulation G, Conditions for Use of Non-GAAP Financial Measures, and other SEC regulations define and prescribe the conditions for use of certain non-GAAP financial information.\nOur measures of free cash flows and the effect of foreign exchange rates on our consolidated statements of operations meet the definition of non-GAAP financial measures.', 'The functional currency of our subsidiaries that either operate or support these stores is generally the same as the local currency.\nAssets and liabilities of these subsidiaries are translated into U.S.\nDollars at period-end foreign exchange rates, and revenues and expenses are translated at average rates prevailing throughout the period.', 'The U.S.\ntax rules also provide for enhanced accelerated depreciation deductions by allowing the election of full expensing of qualified property, primarily equipment, through 2022.\nOur federal tax provision included the election of full expensing of qualified property for 2019 and a partial election for 2020 and 2021.\nThe components of the provision for income taxes, net are as follows (in millions): U.S.', 'We are subject to income taxes in the U.S.\n(federal and state) and numerous foreign jurisdictions.\nSignificant judgment is required in evaluating our tax positions and determining our provision for income taxes.\nDuring the ordinary course of business, there are many transactions and calculations for which the ultimate tax determination is uncertain.\nWe establish reserves for tax-related uncertainties based on estimates of whether, and the extent to which, additional taxes will be due.', 'Dollar and Euro commercial paper programs (the “Commercial Paper Programs”) under which we may from time to time issue unsecured commercial paper up to a total of $10.0 billion (including up to €3.0 billion) at the date of issue, with individual maturities that may vary but will not exceed 397 days from the date of issue.', 'Intangible assets acquired in a business combination that are in-process and used in research and development activities are considered indefinite-lived until the completion or abandonment of the research and development efforts.\nOnce the research and development efforts are completed, we determine the useful life and begin amortizing the assets.\nAmortization expense for acquired finite-lived intangibles was $565 million, $509 million, and $512 million in 2019, 2020, and 2021.', 'Interest expense was $1.6 billion and $1.8 billion in 2020 and 2021 and was primarily related to debt and finance leases.\nOur long-term lease liabilities were $52.6 billion and $67.7 billion as of December 31, 2020 and 2021.\nOur long-term debt was $31.8 billion and $48.7 billion as of December 31, 2020 and 2021.\nSee Item 8 of Part II, “Financial Statements and Supplementary Data — Note 4 — Leases and Note 6 — Debt” for additional information.'], 'metadata': [{'full_metadata': 'AMZN_2022_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2022_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2022_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2022_MANAGEMENT_DISCUSSION_10-K'}, {'full_metadata': 'AMZN_2022_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2022_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2022_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2022_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2022_FINANCIAL_STATEMENTS_10-K'}, {'full_metadata': 'AMZN_2022_MANAGEMENT_DISCUSSION_10-K'}]}})

print(" ".join(relevant_dict['AMZN_2021']["docs"]))

In arrangements where we have multiple performance obligations, the transaction price is allocated to each performance obligation using the relative stand-alone selling price.
We generally determine stand-alone selling prices based on the prices charged to customers or using expected cost plus a margin.
A description of our principal revenue generating activities is as follows: We offer consumer products through our online and physical stores. Our total long-term debt obligations are as follows (in millions): Unamortized discount and issuance costs, net The weighted average remaining lives of the 2012, 2014, 2017, and 2020 Notes were 1.9, 11.8, 16.2, and 18.7 years as of December 31, 2020.
The combined weighted average remaining life of the Notes was 15.8 years as of December 31, 2020.
Interest on the Notes is payable semi-annually in arrears. Amounts held in foreign currencies were $15.3 billion and $23.5 billion as of December 31, 2019 and 2020, and were primarily Euros, British Poun

In [5]:
m = 'AAPL_2022_RISK_FACTORS_10-K'
s = 'AAPL_2022_BUSINESS_10-K'

" ".join(m.split("_")[2:-1])

'RISK FACTORS'